In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import pickle
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score, precision_recall_curve, auc, ConfusionMatrixDisplay, PrecisionRecallDisplay, classification_report, roc_auc_score
from scikitplot.metrics import plot_roc

In [2]:
# Creo una clase para poder utilizar los estimadores en un gridsearch
class ClassifierWrapper(BaseEstimator):
    """
    A Custom Wrapper that can switch between classifiers.
    """ 
    def __init__(self, estimator=None):
        self.estimator = estimator

    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self

    def predict(self, X, y=None):
        return self.estimator.predict(X)

    def predict_proba(self, X):
        return self.estimator.predict_proba(X)

    def score(self, X, y):
        return self.estimator.score(X, y)

In [3]:
location = "./WA_Fn-UseC_-Telco-Customer-Churn.csv"
pd.set_option("display.max_colwidth",200)
df = pd.read_csv(location,encoding = "utf-8")

In [4]:
def limpieza_df(data):
        continuas = ['tenure','monthlycharges','totalcharges']
        discretas = ['gender', 'seniorcitizen', 'partner', 'dependents', 
        'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity',
        'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv',
        'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod',
        'churn'] 
        customer_id = []
        data.columns= data.columns.str.lower()
        for i in continuas:
            data.drop(data[data[i]==' '].index,inplace=True)
            data[i] = data[i].apply(lambda x: float(x))
        customer_id = data.customerid
        data.drop(columns=['customerid'],axis=1,inplace=True)
        df_dummies = pd.get_dummies(data,drop_first=True)
        df_dummies.columns = df_dummies.columns.str.lower()
        df_dummies = pd.DataFrame(df_dummies)
        customerid =pd.DataFrame(customer_id)
        df_dummies = customerid.join(df_dummies,how='left')
        df_dummies_internet1 = df_dummies.loc[(df_dummies['internetservice_no'] == 0)]
        return df_dummies_internet1

In [5]:
data = limpieza_df(df)

In [6]:
data.drop(columns=['multiplelines_no phone service','onlinesecurity_no internet service','onlinebackup_no internet service','deviceprotection_no internet service','techsupport_no internet service','streamingtv_no internet service','streamingmovies_no internet service'],inplace=True)

In [7]:
# data_df = df.to_json()

In [8]:
# r = requests.get("http://127.0.0.1:5000/data",json=data_df)

In [9]:
# r=_

In [10]:
# r

In [11]:
# Cambiando el numero se cambia el ID cliente
numero_ID = data.customerid.iloc[9]
numero_ID

'6388-TABGU'

In [12]:
# for i,element in enumerate(r.customerid):
#     if element == numero_ID: 
#         variable = r[i:i+1].copy()
#         print(type(r[i:i+1]))
        

In [13]:
def prediccion(numero_ID):
    grilla_usuarios = frozenset(data.customerid)

    if numero_ID in grilla_usuarios:
#         model.fit(X_train,y_train)
        features = data[numero_ID == data.customerid].drop(columns=['customerid','churn_yes']).values.tolist()
        #         print(type({"valores":[feat.tolist()]}))
#         print(numero_ID)
#         r = requests.get("http://10.1.8.31:5020/predict",json={"valores":features.tolist()})
        try:
            print(features)
            result = requests.get("http://127.0.0.1:5000/predict",json={"valores":features}).json()
            return result
        except:
            print('something is wrong')
#     r.json()
    else:
        return print('no es un usuario registrado')

In [18]:
# Cambiando el numero se cambia el ID cliente
numero_ID = data.customerid.iloc[150]
numero_ID

'8017-UVSZU'

In [23]:
lista_ID = []
for i in range(150):
    lista_ID.append(data.customerid.iloc[i])

In [24]:
lista_ID

['7590-VHVEG',
 '5575-GNVDE',
 '3668-QPYBK',
 '7795-CFOCW',
 '9237-HQITU',
 '9305-CDSKC',
 '1452-KIOVK',
 '6713-OKOMC',
 '7892-POOKP',
 '6388-TABGU',
 '9763-GRSKD',
 '8091-TTVAX',
 '0280-XJGEX',
 '5129-JLPIS',
 '3655-SNQYZ',
 '9959-WOFKT',
 '4190-MFLUW',
 '4183-MYFRB',
 '8779-QRDMV',
 '3638-WEABW',
 '6322-HRPFA',
 '6865-JZNKO',
 '6467-CHFZW',
 '8665-UTDHZ',
 '5248-YGIJN',
 '8773-HHUOZ',
 '3841-NFECX',
 '4929-XIHVW',
 '6827-IEAUQ',
 '3413-BMNZE',
 '6234-RAAPL',
 '6047-YHPVI',
 '6572-ADKRS',
 '5380-WJKOV',
 '8168-UQWWF',
 '8865-TNMNX',
 '9489-DEDVP',
 '4671-VJLCL',
 '4080-IIARD',
 '3714-NTNFO',
 '5948-UJZLF',
 '7760-OYPDY',
 '7639-LIAYI',
 '2954-PIBKO',
 '8012-SOUDQ',
 '9420-LOJKX',
 '6575-SUVOI',
 '7495-OOKFY',
 '4667-QONEA',
 '1658-BYGOY',
 '8769-KKTPH',
 '5067-XJQFU',
 '5954-BDFSG',
 '0434-CSFON',
 '1215-FIGMP',
 '0526-SXDJP',
 '0557-ASKVU',
 '5698-BQJOH',
 '5122-CYFXA',
 '8627-ZYGSZ',
 '3410-YOQBQ',
 '7410-OIEDU',
 '2273-QCKXA',
 '1891-QRQSA',
 '5630-AHZIL',
 '2673-CXQEU',
 '6416-JNV

In [36]:
for elemento in predictions:
    print(elemento.values())

dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values([0.0])
dict_values(

In [25]:
predictions = []
for i in range(len(lista_ID)):
    predictions.append(prediccion(lista_ID[i]))

[[0.0, 1.0, 29.85, 29.85, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]]
[[0.0, 34.0, 56.95, 1889.5, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
[[0.0, 2.0, 53.85, 108.15, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]]
[[0.0, 45.0, 42.3, 1840.75, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[0.0, 2.0, 70.7, 151.65, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]]
[[0.0, 8.0, 99.65, 820.5, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]]
[[0.0, 22.0, 89.1, 1949.4, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0]]
[[0.0, 10.0, 29.75, 301.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]]
[[0.0, 28.0, 104.

In [16]:
# Si coloco un numero de usuario inexistente pasa lo siguiente
numero_ID_incorrecto='1111-INVENTADO'

In [17]:
prediccion(numero_ID_incorrecto)

no es un usuario registrado
